In [ ]:
import tensorflow as tf
from tensorflow import keras

# Convolutional Neural Networks

In the convolutional layer, the neurons are not connected to every part of the input data.
The anatomy of a Conv layer



![CNN](cnn_architecture.png)

When the input a single dimension array (vector, time series), use a Conv1D layer.
https://keras.io/api/layers/convolution_layers/convolution1d/


When the input is a 2D image (2D array) or a time series with multiple channels (example of a seismogram with 3 components), use Conv2D layers.
https://keras.io/api/layers/convolution_layers/convolution2d/

A neuron weight can be represented as a small image the size of the receiptive field. It reduces the dimensionality of the image.
Filters are convolutional kernels.
Stride is the shift over features
Input_shape is the dimension of the data.


## Pooling layers


In [44]:
model = keras.Sequential()
# Must define the input shape in the first layer of the neural network
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
# Take a look at the model summary
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 64)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        8224      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 1568)            

In [45]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [77]:
# (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
# print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
class_names = ["tshirt","trousers","pullover","dress","coat","sandal","shirt","sneaker","bag","boot"]
X_val,X_train = X_train_full[:5000]/255.0,X_train_full[5000:]/255.0
y_val,y_train = y_train_full[:5000],y_train_full[5000:]
# X_train = X_train.reshape( 28, 28, -1)
# y_train = y_train.reshape( 1, -1)
X_train=X_train[...,None]
print(X_train.shape)
print(y_train.shape)

(55000, 28, 28, 1)
(55000,)


In [78]:
model.fit(X_train,
         y_train,
         batch_size=64,
         epochs=10,
         validation_data=(X_val,y_val),
         callbacks=[checkpoints_cb])

Epoch 1/10


ValueError: in user code:

    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _

In [79]:
# Evaluate the model on test set
score = model.evaluate(x_test, y_test, verbose=0)
# Print test accuracy
print('\n', 'Test accuracy:', score[1])

ValueError: in user code:

    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1233 test_function  *
        return step_function(self, iterator)
    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1224 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/marinedenolle/opt/anaconda3/envs/uwdsgeo/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.xlabel('epochs')

In [1]:
import keras
from keras import layers, models, regularizers
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import to_categorical
# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 6} ) 
# sess = tf.Session(config=config) 
# keras.backend.set_session(sess)

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

## Example on seismic data

In this class, we will use a simplified version of ConvNetQuake (Perol et al, 2018).



![ConvNetQuake](fig2.jpg)

In [ ]:
# load OK029 template data:
with h5py.File("./data/templates_029.h5", "r") as f:
    crap1 = np.asarray(f['earthquakes'])
    crap2=np.asarray(f["noise"])

    # load OK027 template data:
with h5py.File("./data/templates_027.h5", "r") as f:
    crap1 = np.asarray(f['earthquakes'])
    crap2=np.asarray(f["noise"])

In [ ]:
# make training data
quakes=np.zeros(shape=(len(crap1[:,1,1]),1000,3),dtype=np.float32)
noise=np.zeros(shape=(len(crap1[:,1,1]),1000,3),dtype=np.float32)
quakes2=np.zeros(shape=(len(crap2[:,1,1]),1000,3),dtype=np.float32)
noise2=np.zeros(shape=(len(crap2[:,1,1]),1000,3),dtype=np.float32)
for iq in range(len(quakes[:,1,1])):
    for ic in range(3):
        if np.max(np.abs(crap1[iq,ic,:]))>0:
            quakes[iq,:,ic]=crap1[iq,ic,:]/np.max(np.abs(crap1[iq,ic,:]))
        if np.max(np.abs(crap2[iq,ic,:]))>0:
            noise[iq,:,ic]=crap2[iq,ic,:]/np.max(np.abs(crap2[iq,ic,:]))

for iq in range(len(crap1[:,1,1])):
    for ic in range(3):
        if np.max(np.abs(crap1[iq,ic,:]))>0:
            quakes2[iq,:,ic]=crap1[iq,ic,:]/np.max(np.abs(crap1[iq,ic,:]))
        if np.max(np.abs(crap2[iq,ic,:]))>0:
            noise2[iq,:,ic]=crap2[iq,ic,:]/np.max(np.abs(crap2[iq,ic,:]))
iq1=np.where( ( np.abs(quakes[:,0,0])>0)&(np.isfinite(quakes[:,0,0])))[0]
iq2=np.where( (np.abs(quakes2[:,0,0])>0)&(np.isfinite(quakes2[:,0,0])))[0]
print('done preparing the training data')

In [ ]:
# label data
train_labels = np.concatenate((np.ones(len(iq1)+len(iq2),dtype=np.int),np.zeros(len(iq1)+len(iq2),dtype=np.int)))#+noise3.shape[0])))
train_waves = np.zeros(shape=(len(train_labels),1000,3,1))
train_waves[:,:,:,0] = np.concatenate((quakes[iq1,:,:],quakes2[iq2,:,:],noise[iq1,:,:],noise2[iq2,:,:]),axis=0)
y_binary = to_categorical(train_labels)
indx=np.random.choice(len(train_labels),len(train_labels))

# train data is 80% of the entire data set
ntrain = np.arange(0,round(0.8*len(indx)))
ntest = np.arange(ntrain[-1]+1,len(indx))
print("number of training samples",len(ntrain),"number of test tests",len(ntest))

In [ ]:
l2, cats = 0.00001,2 # learning rate, number of classes output

# building the architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1000,3,1), use_bias=True, strides=(2,2), kernel_regularizer=regularizers.l2(l2), padding='SAME'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', use_bias=True, strides=(2,2), kernel_regularizer=regularizers.l2(l2), padding='SAME'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', use_bias=True, strides=2, kernel_regularizer=regularizers.l2(l2), padding='SAME'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', use_bias=True, strides=2, kernel_regularizer=regularizers.l2(l2), padding='SAME'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', use_bias=True, strides=2, kernel_regularizer=regularizers.l2(l2), padding='SAME'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', use_bias=True, strides=2, kernel_regularizer=regularizers.l2(l2), padding='SAME'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', use_bias=True, strides=2, kernel_regularizer=regularizers.l2(l2), padding='SAME'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', use_bias=True, strides=2, kernel_regularizer=regularizers.l2(l2), padding='SAME'))
model.add(layers.Flatten())
model.add(layers.Dense(cats, activation='softmax'))
# compile the network
model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
# training
history=model.fit(train_waves[indx[ntrain],:,:,:], y_binary[indx[ntrain]],validation_split=0.2, epochs=20, batch_size=128,shuffle=True)
# save the weights
weights1=model.get_weights()

print(len(weights1))

# testing
test=model.evaluate(train_waves[indx[ntest],:,:,:], y_binary[indx[ntest]],batch_size=64,verbose=1)
print('loss and accuracy at test')
print(test)

In [ ]:

# check for overfitting
history_dic = history.history
loss_values = history_dic['loss']
val_loss_values = history_dic['val_loss']
acc_values = history_dic['acc']
val_acc_values = history_dic['val_acc']
epochs=range(1,len(loss_values)+1)
fig,ax=plt.subplots(2)
ax[0].plot(epochs,loss_values,'bo',label='Training loss')
ax[0].plot(epochs,val_loss_values,'b',label='Validation loss')
ax[0].set_title('Training and validation loss')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].legend()
ax[1].plot(epochs,acc_values,'bo',label='Training accuracy')
ax[1].plot(epochs,val_acc_values,'b',label='Validation accuracy')
ax[1].set_title('Training and validation accuracy')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('accuracy')
ax[1].legend()
plt.show()


In [ ]:
# load long data:
with h5py.File("/content/drive/My Drive/EPS268-ML/TUTORIALS/Quake_detection_week3/OK027_july_2014.h5", "r") as f:
    crap1 = np.asarray(f['seismic-data'])
print(crap1.shape)
plt.plot(crap1[:,10000:12000].transpose())
print(crap1.shape)

In [ ]:
# here you want to slide through the data.
twin= 10 # seconds
sps = 100 # 100Hz sampling rate
stepsize = 2 # every 2 seconds
nwin = int(twin*sps)
nsps = int(stepsize*sps)
i=0
while i < crap1.shape[1]:
  print(i/100,(i+nwin)/100)
#   window0 = crap1[0, i:i+nwin]
  model.predict(crap1[:,i:i+nwin])
  
  
  exit()
  
  i+=nsps